In [1]:
!pip install pandas
!pip install cplex
!pip install datetime

In [2]:
# __________________________________ ALL REQUIRED IMPORTS __________________________________

import sqlite3
import pandas as pd
from datetime import datetime

In [3]:
# __________________________________ LOADING DATA INTO A DATABASE __________________________________

# URLs for data sources
DATA_URLS = {

    "inventory": "https://raw.githubusercontent.com/team79interiit/qcc_data_files/main/inv.csv",
    "schedule": "https://raw.githubusercontent.com/team79interiit/qcc_data_files/main/sch.csv",
    "pnr_booking": "https://raw.githubusercontent.com/team79interiit/qcc_data_files/main/pnrb.csv",
    "pnr_passenger": "https://raw.githubusercontent.com/team79interiit/qcc_data_files/main/pnrp.csv",
    "pnr_booking_template": "https://raw.githubusercontent.com/team79interiit/qcc_data_files/main/booking_template.csv",

}

DATABASE = "data.db"

In [4]:
# Function to load data from URLs into dataframes
def load_data(data_urls=dict()):

    data_frames = {}

    for key, url in data_urls.items():

        data_frames[key] = pd.read_csv(url)

    return data_frames

In [5]:
# Function to add a DataFrame as a table to a SQLite database
def add_table_to_database(dataframe, tablename=''):

    try:

        # Establish connection to SQLite database
        connection = sqlite3.connect(DATABASE)

        # Add DataFrame as a table to the database
        dataframe.to_sql(tablename, connection, if_exists='fail', index=False)
        print(f"Table '{tablename}' added to the database.")

    except Exception as e:

        # Handle case where table already exists in the database
        print(f"{e}")

    finally:

        # Ensure the connection is closed after operations
        connection.commit()
        connection.close()

In [6]:
# Load data frames from URLs
data_frames = load_data(DATA_URLS)

# Select specific tables to add to the database
tables_to_add = {

    'inv': data_frames['inventory'],
    'pnrb': data_frames['pnr_booking'],
    'pnrp': data_frames['pnr_passenger'],
}

# Add selected tables to the database
for table_name, df in tables_to_add.items():
  add_table_to_database(df, table_name)

Table 'inv' already exists.
Table 'pnrb' already exists.
Table 'pnrp' already exists.


In [7]:
#  ___________________ WE ARE ASSUMING THAT THE FOLLOWING MINIMAL INFO IS GIVEN ON CHANGES IN SCHEDULE ______________

# Define constants for types of changes
CANCELLED = "Cancelled"
DELAYED = "Delayed"

# List of available types of changes
type_of_changes = [CANCELLED, DELAYED]

# Information about changes in flight schedules
changes = [
    {
        "Flight Number": 2504,
        "Departure Date": '4/3/2024',
        "Type of change": CANCELLED,
    },
    {
        "Flight Number": 3723,
        "Departure Date": "04/07/2024",
        "Type of change": DELAYED,
        "New Departure Date Time": "2024-05-05 11:52:00",
        "New Arrival Date Time": "2024-05-05 20:30:00",
    }
]

# Since the date & time in database are inconsistent we are using a global data structre to keep track of format while writing code
FORMATS = {
    "inv":[{
        "DepartureDate" : "%m/%d/%Y",
        "DepartureDateTime": "%Y-%m-%d %H:%M:%S",
        "ArrivalDateTime": "%Y-%m-%d %H:%M:%S",
    }, True], # True indicates that there is 0 infront of single digit numbers in dates

    "pnrb":[{
        "DEP_DT" : "%m/%d/%Y",
        "DEP_DTML" : "%m/%d/%Y %H:%M",
        "ARR_DTML": "%m/%d/%Y %H:%M",
        "DEP_DTMZ": "%m/%d/%Y %H:%M",
        "ARR_DTMZ": "%m/%d/%Y %H:%M",
    }, False], # False indicates that there is no 0 infront of single digit numbers in dates
}

In [8]:
def search_for_affected_flight_inv(flight_number, departure_date):

  connection = sqlite3.connect(DATABASE)
  cursor = connection.cursor()

  sql_search_query = '''
  SELECT * FROM inv
  WHERE FlightNumber = ? and DepartureDate = ?;
  '''

  # The following code will make sure the departure_date is in correct format & also adds 0's if single digit date
  try:
    departure_date = datetime.strptime(departure_date, "%m/%d/%Y").strftime("%m/%d/%Y")
  except Exception as e:
    print("departure date is not in %m/%d/%Y format")

  cursor.execute(sql_search_query,(flight_number, departure_date))
  results = cursor.fetchall()

  connection.commit()
  connection.close()
  return results

In [9]:
def search_for_affected_pnrb(flight_number, departure_date):

    connection = sqlite3.connect(DATABASE)
    cursor = connection.cursor()

    sql_search_query = '''
    SELECT * FROM pnrb
    WHERE FLT_NUM = ? AND DEP_DT = ?;
    '''

    # Attempt to convert departure_date to the correct format
    try:
        departure_date = datetime.strptime(departure_date, "%m/%d/%Y").strftime("%m/%d/%Y")
    except Exception as e:
        print("Departure date is not in the format %m/%d/%Y.")

    # Remove leading zeros from single-digit days or months
    departure_date_parts = departure_date.split('/')
    departure_date = '/'.join(str(int(part)) for part in departure_date_parts)

    cursor.execute(sql_search_query, (flight_number, departure_date))
    results = cursor.fetchall()

    connection.commit()
    connection.close()
    return results

In [10]:
def update_inv_with_newdatetime (new_dept_time='', new_arr_time='', inv_id=''):

    connection = sqlite3.connect(DATABASE)
    cursor = connection.cursor()

    # Using parameterized query to avoid SQL injection
    sql_update_query = '''
    UPDATE inv
    SET "DepartureDateTime" = ?, "ArrivalDateTime" = ?
    WHERE InventoryId = ? '''

    # Attempt to convert departure_date to the correct format
    try:

        new_dept_time = datetime.strptime(new_dept_time, "%Y-%m-%d %H:%M:%S").strftime("%Y-%m-%d %H:%M:%S")

    except Exception as e:

        print("new_dept_time is not in the format %Y-%m-%d %H:%M:%S")

    # Attempt to convert departure_date to the correct format
    try:

        new_arr_time = datetime.strptime(new_arr_time, "%Y-%m-%d %H:%M:%S").strftime("%Y-%m-%d %H:%M:%S")

    except Exception as e:

        print("new_arr_time is not in the format %Y-%m-%d %H:%M:%S")

    # Tuple of values to be substituted into the query
    values = (new_dept_time, new_arr_time, inv_id)

    cursor.execute(sql_update_query, values)
    connection.commit()
    connection.close()

In [11]:
def add_affected_passengers_to_database(inventory_id, flight_number, departure_date):

  pnr_booking_template_df = pd.read_csv(DATA_URLS["pnr_booking_template"])
  connection = sqlite3.connect(DATABASE)
  cursor = connection.cursor()

  # Identify the affected passengers and pass it on to post process
  # Step 1 : Create a table with Inventory Id of the affected flight as table name
  try:

    table_name = inventory_id.replace('-','_')
    pnr_booking_template_df.to_sql(table_name, connection, if_exists='replace', index=False)

    # Step 2 : Add the affected passengers to the table
    affected_pnrs = search_for_affected_pnrb(flight_number, departure_date)

    for pnr in affected_pnrs:

      insert_data_query = f'''
      INSERT INTO  {table_name}
      VALUES {pnr}
      '''
      cursor.execute(insert_data_query)

  except Exception as e:

    print(f"HURRAY! -- The passengers of the flight on this Inventory entry are already identified | {e} ")

  connection.commit()
  connection.close()
  return

In [12]:
affected_flights = dict()

for change in changes:

  flight_number = change["Flight Number"]
  departure_date = change["Departure Date"]
  type_of_change = change["Type of change"]

  search_result = search_for_affected_flight_inv(flight_number, departure_date)

  if search_result == []:

    print(f" OOPS! ----- No Inventory entry exists for the given change : {change} ------ ")
    continue

  search_result = search_result[0]

  # Preparing to handling the change in schedule
  # dictionary to pass the information to the quantum solver

  inventory_id = search_result[0]

  if type_of_change in type_of_changes:

    add_affected_passengers_to_database(inventory_id, flight_number, departure_date)

    if type_of_change == CANCELLED:

      affected_flights[inventory_id] = type_of_change

    elif type_of_change == DELAYED:

      affected_flights[inventory_id] = type_of_change

      new_departure_date_time = change["New Departure Date Time"]
      new_arrival_date_time = change["New Arrival Date Time"]

      update_inv_with_newdatetime(new_departure_date_time, new_arrival_date_time, inventory_id)

    else:

      print(f"----- Unidentified type of change : {type_of_change} ------")

# print(affected_flights)

In [13]:
# __________________________________ RULE ENGINE __________________________________

MAX_ETD_FROM_IMPACTED_FLIGHT_IN_HOURS = 72
MAX_CONNECTION_TIME = 12
MIN_CONNECTION_TIME = 1

# This reference table can be used for identifying the cabin for each class of service
CABIN_CLASS_MAP = {
    "J": ["A", "D", "J"],
    "F": ["F", "B"],
    "Y": ["S", "V", "W", "Z", "O", "S", "T", "U", "M", "N", "Y"]
}

# This will be Used when Original class is not available to rebook
CLASS_TO_CLASS_MAP = {
    "A": ["F", "S"],
    "S": ["F", "A"],
    "F": ["S", "A"],
    "J": ["O", "C", "I"],
    "C": ["O", "J", "I"],
    "I": ["J", "C", "O"],
    "O": ["J", "C", "I"],
    "Y": ["B", "P", "Z"],
    "B": ["Y", "P", "Z"],
}

# 1. This is only applicable if Downgrade is on.
# 2. If Downgrade to Cabin J/F then refer to Downgrade class respectively
DOWNGRADE_CLASS_MAP = {
    "J": "Y",
    "F": "Y",
}

# 1. This is only applicable if Upgrade is enabled.
# 2. If Upgraded to Cabin F/J then refer to Upgrade class respectively
UPGRADE_CLASS_MAP = {
    "Y": "F",
    "F": "J",
}

class PNR:
    def __init__(self):

        self.Recloc = None
        self.TYPE = None
        self.SSR = None
        self.Cabin = None
        self.Class = None
        self.Number_Of_Down_lineConnections = 0
        self.PaidServices = None
        self.Booked_As = None
        self.Number_of_PAX = None
        self.Loyalty = None
        self.ForceKickOut = None

    def Score(self):

        Special_Services = ["INFT", "WHCR", "WCHS", "WCHC", "LANG", "CHILD", "EXST", "BLND", "DEAF"]
        total_score = None
        self.ForceKickOut = True

        # SSR Score
        if self.TYPE == "PNR.INDIVIDUAL" and self.SSR in Special_Services:
            total_score += 200

        # Cabin Score
        if self.TYPE == "PNR.INDIVIDUAL" and self.Cabin:
            cabin_score_mapping = {
                "J": 2000,
                "F": 1700,
                "Y": 1500
            }
            total_score += cabin_score_mapping.get(self.Cabin, 0)

        # Class Score
        if self.TYPE == "PNR.INDIVIDUAL" and self.Class:
            class_score_mapping = {
                "A": 1000,
                "C": 700,
                "K": 500,
            }
            total_score += class_score_mapping.get(self.Class, (0, 0))[0]

        # Connection Score
        if self.TYPE == "PNR.INDIVIDUAL" and self.Number_Of_Down_lineConnections is not None:
            total_score += 100 * self.Number_Of_Down_lineConnections

        # Paid Service Score
        if self.TYPE == "PNR.INDIVIDUAL" and self.PaidServices == "Yes":
            self.ForceKickOut = False
            total_score += 200

        # Booking-Type Score
        if self.TYPE == "PNR.INDIVIDUAL" and self.Booked_As == "Group":
            self.ForceKickOut = False
            total_score += 500

        # No of PAX Score
        if self.TYPE == "PNR.INDIVIDUAL" and self.Number_of_PAX is not None:
            self.ForceKickOut = False
            total_score += 50 * self.Number_of_PAX


In [14]:
def get_affected_pnrb (table_name):

  connection = sqlite3.connect(DATABASE)
  cursor = connection.cursor()

  sql_search_query = f'''
  SELECT * FROM {table_name}
  '''

  cursor.execute(sql_search_query)
  results = cursor.fetchall()

  connection.commit()
  connection.close()
  return results

In [15]:
flights = list(affected_flights.keys())
for flight in flights:
  flight_data = get_affected_pnrb(flight.replace('-', '_'))
  print(flight_data)

[('DRGS80', '3/2/2024 16:35', 'ZZ20240403BLRCCU2504', 'GN', 'FirstClass', '1', '4', '3', 'ZZ', '2504', 'BLR', 'CCU', '4/3/2024', '4/3/2024 15:35', '4/3/2024 18:17', '4/3/2024 10:05', '4/3/2024 12:47'), ('YEZQ47', '3/2/2024 16:35', 'ZZ20240403BLRCCU2504', 'GN', 'FirstClass', '1', '2', '4', 'ZZ', '2504', 'BLR', 'CCU', '4/3/2024', '4/3/2024 15:35', '4/3/2024 18:17', '4/3/2024 10:05', '4/3/2024 12:47'), ('JDDM40', '3/2/2024 16:35', 'ZZ20240403BLRCCU2504', 'GN', 'FirstClass', '1', '3', '2', 'ZZ', '2504', 'BLR', 'CCU', '4/3/2024', '4/3/2024 15:35', '4/3/2024 18:17', '4/3/2024 10:05', '4/3/2024 12:47'), ('AVFF16', '3/2/2024 16:35', 'ZZ20240403BLRCCU2504', 'GN', 'FirstClass', '1', '4', '1', 'ZZ', '2504', 'BLR', 'CCU', '4/3/2024', '4/3/2024 15:35', '4/3/2024 18:17', '4/3/2024 10:05', '4/3/2024 12:47'), ('DBLE79', '3/2/2024 16:35', 'ZZ20240403BLRCCU2504', 'GN', 'FirstClass', '1', '2', '2', 'ZZ', '2504', 'BLR', 'CCU', '4/3/2024', '4/3/2024 15:35', '4/3/2024 18:17', '4/3/2024 10:05', '4/3/2024 12: